In [11]:
import gym
import holdem
import agent

In [12]:
def add_users(env):
    model_list = list()

    # start with 4 players
    env.add_player(0, stack=5000) # add a player to seat 0 with 1000 "chips"
#     model_list.append(agent.allFoldModel())
    model_list.append(agent.MontecarloModel())

    env.add_player(1, stack=5000) # add another player to seat 1 with 2000 "chips"
#     model_list.append(agent.allFoldModel())
    model_list.append(agent.MontecarloModel())

    env.add_player(2, stack=5000) # add another player to seat 2 with 3000 "chips"
    # model_list.append(agent.allFoldModel())
    model_list.append(agent.MontecarloModel())

    env.add_player(3, stack=5000) # add another player to seat 3 with 1000 "chips"
    model_list.append(agent.allFoldModel())

    env.add_player(4, stack=5000) # add another player to seat 3 with 1000 "chips"
    model_list.append(agent.dqnModel())

    env.add_player(5, stack=5000) # add another player to seat 3 with 1000 "chips"
    model_list.append(agent.dqnModel())
#     model_list.append(agent.allFoldModel())

    # env.add_player(6, stack=1000) # add another player to seat 3 with 1000 "chips"
    # model_list.append(agent.allCallModel())

    # env.add_player(7, stack=1000) # add another player to seat 3 with 1000 "chips"
    # model_list.append(agent.MontecarloModel())

    # env.add_player(8, stack=1000) # add another player to seat 3 with 1000 "chips"
    # model_list.append(agent.allFoldModel())

    # env.add_player(9, stack=1000) # add another player to seat 3 with 1000 "chips"
    # model_list.append(agent.allinModel())

    return model_list

In [13]:
playerid = 4
episodes = 30
pre_train_steps = 20 #How many steps of random actions before training begins.

def lets_play():
    env = gym.make('TexasHoldem-v2') # holdem.TexasHoldemEnv(2)

    model_list = add_users(env)
    
    agent = model_list[playerid]
    total_steps = 0

    # Iterate the game
    for e in range(episodes):
    #     lets_play(env, env.n_seats, model_list)
        # reset state in the beginning of each game
        state = env.reset()
        env.render(mode='human')
        done = False
        if env.episode_end:
            break
        stack = state.player_states[playerid].stack
        # time_t represents each frame of the game
        # Our goal is to keep the pole upright as long as possible until score of 500
        # the more time_t the more score
        while not done:
            # play safe actions, check when no one else has raised, call when raised.
            # print(">>> Debug Information ")
            # print("state(t)")
            # for p in cur_state.player_states:
            #     print(p)
            # print(cur_state.community_state)

            actions = holdem.model_list_action(state, n_seats=env.n_seats, model_list=model_list)
            next_state, rewards, done, _ = env.step(actions)
            action = actions[playerid][0]
            reward = 0
            if state.community_state.current_player == playerid:
                agent.remember(state, action, reward, next_state, done, playerid)
#                 print('actions: ', action)
                agent.onlineTrainModel()
                env.render(mode='human')
            state = next_state
#             print("memory len: ", len(agent.memory))

    #         print('table card: ', state.community_card)

            # print("action(t), (CALL=1, RAISE=2, FOLD=3 , CHECK=0, [action, amount])")
            # print(actions)
            if done:
                # print the score and break out of the loop
                print("episode: {}/{}, score: {}".format(e, episodes, reward))
                reward = rewards[playerid] - stack
                stack = rewards[playerid]
                agent.memory.pop()
                agent.remember(state, action, reward, next_state, done, playerid)

            # print("reward(t+1)")
            # print(rews)
            # print("<<< Debug Information ")

            # print("final state")
            # print(cur_state)

            # total_stack = sum([p.stack for p in env._seats])
            # if total_stack != 10000:
            #     return

            # train the agent with the experience of the episode
        if total_steps > pre_train_steps:
            agent.replay(10)

        total_steps += 1

        agent.saveModel()

        print("Episode End!!!")



In [14]:
for i in range(30):
    print(i, " plays")
    try:
        lets_play()
    except:
        print('Error!!!!')

0  plays
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_91 (Dense)             (None, 100)               36800     
_________________________________________________________________
dense_92 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_93 (Dense)             (None, 4)                 204       
Total params: 42,054
Trainable params: 42,054
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_94 (Dense)             (None, 100)               36800     
_________________________________________________________________
dense_95 (Dense)             (None, 50)                5050      
_______________________________________________________________

Episode End!!!
Cycle 2, total pot: 0 >>>
last action by player 1:	_ check
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [K♦],[A♦] stack: 0
1 [5♥],[5♣] stack: 0
2 [J♦],[9♥] stack: 21020
3 [Q♦],[T♥] stack: 5000
4 [2♠],[2♣] stack: 3980
5 [3♦],[Q♣] stack: 0
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
2  plays
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_115 (Dense)            (None, 100)               36800     
_________________________________________________________________
dense_116 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_117 (Dense)            (None, 4)                 204       
Total params: 42,054
Trainable params: 42,054
Non-trainable params: 0
_________________________________________________________________
_______________________________________________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_145 (Dense)            (None, 100)               36800     
_________________________________________________________________
dense_146 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_147 (Dense)            (None, 4)                 204       
Total params: 42,054
Trainable params: 42,054
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_148 (Dense)            (None, 100)               36800     
_________________________________________________________________
dense_149 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_

Cycle 1, total pot: 30 >>>
last action by player 4:	x fold
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [9♦],[7♠] stack: 5000
1 [K♣],[K♦] stack: 4990
2 [K♥],[3♦] stack: 4980
3 [J♠],[7♦] stack: 5000
4 [K♠],[2♦] stack: 5000
5 [6♣],[4♦] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
episode: 0/30, score: 0
Episode End!!!
Cycle 2, total pot: 0 >>>
last action by player 1:	_ check
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [9♦],[7♠] stack: 0
1 [K♣],[K♦] stack: 20000
2 [K♥],[3♦] stack: 0
3 [J♠],[7♦] stack: 5000
4 [K♠],[2♦] stack: 5000
5 [6♣],[4♦] stack: 0
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
6  plays
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_163 (Dense)            (None, 100)               36800     
_________________________________________________________________
dense_164 (Dense) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_184 (Dense)            (None, 100)               36800     
_________________________________________________________________
dense_185 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_186 (Dense)            (None, 4)                 204       
Total params: 42,054
Trainable params: 42,054
Non-trainable params: 0
_________________________________________________________________
Cycle 1, total pot: 30 >>>
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [A♣],[K♠] stack: 5000
1 [6♣],[Q♣] stack: 4990
2 [A♠],[Q♦] stack: 4980
3 [3♣],[4♥] stack: 5000
4 [2♥],[4♠] stack: 5000
5 [A♥],[J♠] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
Cycle 1, total pot: 5030 >>>
last action by player 4:	^ raise, current bet: 5000
community:
- [

...

Cycle 1, total pot: 50 >>>
last action by player 4:	- call, current bet: 20
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [5♠],[8♦] stack: 5000
1 [4♣],[Q♠] stack: 4990
2 [J♣],[T♠] stack: 4980
3 [T♦],[J♠] stack: 5000
4 [Q♣],[3♠] stack: 4980
5 [6♣],[8♠] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
Cycle 1, total pot: 100 >>>
last action by player 4:	_ check
community:
- [7♦],[A♦],[2♦],[  ],[  ] 
players:
0 [5♠],[8♦] stack: 4980
1 [4♣],[Q♠] stack: 4980
2 [J♣],[T♠] stack: 4980
3 [T♦],[J♠] stack: 5000
4 [Q♣],[3♠] stack: 4980
5 [6♣],[8♠] stack: 4980
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
Cycle 1, total pot: 5080 >>>
last action by player 4:	^ raise, current bet: 4980
community:
- [7♦],[A♦],[2♦],[  ],[  ] 
players:
0 [5♠],[8♦] stack: 4980
1 [4♣],[Q♠] stack: 4980
2 [J♣],[T♠] stack: 4980
3 [T♦],[J♠] stack: 5000
4 [Q♣],[3♠] stack: 0
5 [6♣],[8♠] stack: 4980
6 [  ],[  ] stack: 0
7 [  ],[  ] 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_391 (Dense)            (None, 100)               36800     
_________________________________________________________________
dense_392 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_393 (Dense)            (None, 4)                 204       
Total params: 42,054
Trainable params: 42,054
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_394 (Dense)            (None, 100)               36800     
_________________________________________________________________
dense_395 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_412 (Dense)            (None, 100)               36800     
_________________________________________________________________
dense_413 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_414 (Dense)            (None, 4)                 204       
Total params: 42,054
Trainable params: 42,054
Non-trainable params: 0
_________________________________________________________________
Cycle 1, total pot: 30 >>>
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [A♠],[9♦] stack: 5000
1 [2♠],[J♥] stack: 4990
2 [9♣],[7♣] stack: 4980
3 [J♦],[8♦] stack: 5000
4 [8♠],[4♦] stack: 5000
5 [K♠],[8♣] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
Cycle 1, total pot: 30 >>>
last action by player 4:	x fold
community:
- [  ],[  ],[  ],[  ],[  

Cycle 1, total pot: 30 >>>
last action by player 4:	x fold
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [4♠],[K♠] stack: 5000
1 [2♥],[J♦] stack: 4990
2 [Q♣],[T♦] stack: 4980
3 [6♥],[3♦] stack: 5000
4 [4♦],[4♣] stack: 5000
5 [5♦],[5♥] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
episode: 0/30, score: 0
Episode End!!!
Cycle 2, total pot: 0 >>>
last action by player 1:	_ check
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [4♠],[K♠] stack: 0
1 [2♥],[J♦] stack: 0
2 [Q♣],[T♦] stack: 0
3 [6♥],[3♦] stack: 5000
4 [4♦],[4♣] stack: 5000
5 [5♦],[5♥] stack: 20000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
29  plays
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_439 (Dense)            (None, 100)               36800     
_________________________________________________________________
dense_440 (Dense)

Cycle 3, total pot: 19970 >>>
last action by player 4:	_ check
community:
- [4♠],[3♣],[4♣],[9♣],[2♣] 
players:
0 [8♣],[5♥] stack: 0
1 [7♦],[2♦] stack: 0
2 [J♦],[A♣] stack: 0
3 [4♦],[3♥] stack: 4970
4 [K♠],[3♠] stack: 0
5 [8♦],[Q♦] stack: 5060
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
Cycle 3, total pot: 19970 >>>
last action by player 4:	^ raise, current bet: 0
community:
- [4♠],[3♣],[4♣],[9♣],[2♣] 
players:
0 [8♣],[5♥] stack: 0
1 [7♦],[2♦] stack: 0
2 [J♦],[A♣] stack: 15850
3 [4♦],[3♥] stack: 4970
4 [K♠],[3♠] stack: 4120
5 [8♦],[Q♦] stack: 5060
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
episode: 2/30, score: 0
Episode End!!!
Cycle 4, total pot: 0 >>>
last action by player 4:	^ raise, current bet: 0
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [8♣],[5♥] stack: 0
1 [7♦],[2♦] stack: 0
2 [J♦],[A♣] stack: 15850
3 [4♦],[3♥] stack: 4970
4 [K♠],[3♠] stack: 4120
5 [8♦],[Q♦] stack: 5060
6 [  ],[  ] st